In [ ]:
import mxnet as mx
from mxnet import gluon, nd
import gluonnlp as nlp

train_dataset = nlp.data.TextLineDataset('mimicdemo.train.txt', encoding='utf8')
test_dataset = nlp.data.TextLineDataset('mimicdemo.test.txt', encoding='utf8')

print('#training samples={:d}, #testing samples={:d}'.format(len(train_dataset), len(test_dataset)))
print(train_dataset[0])

import nltk
moses_tokenizer = nlp.data.SacreMosesTokenizer()

spacy_tokenizer = nlp.data.SpacyTokenizer()

dataset = nlp.data.CorpusDataset(
    'mimicdemo.train.txt',
#     sample_splitter=nltk.tokenize.sent_tokenize,
    tokenizer=spacy_tokenizer
#     flatten=False,
#     eos='<eos>'
    )

print('# sentences:', len(dataset))
for sentence in dataset[:50]:
    print('# tokens:', len(sentence), sentence[:100])
    

In [ ]:
import re
train_dataset = nlp.data.TextLineDataset('mimicdemo.train.txt', encoding='utf8')

for index in range(0, len(train_dataset) - 1):
    sentence = train_dataset[index]
    sentence = re.sub(r'[^\w\s]', '', sentence, re.UNICODE)
    sentence = re.sub(r'\b(\w+)( \1\b)+', r'\1', sentence)
    sentence = sentence.lower()
    print(sentence)
    

In [ ]:
import spacy

def cleantext(filename):
    textlinedataset = nlp.data.TextLineDataset(filename + '.txt', encoding='utf8')
    file = open(filename + '.clean.txt','w+') 
    for i in range(0, len(textlinedataset)):
        line = textlinedataset[i]
#         line = remove_between_square_brackets(textlinedataset[i])
#         line = replace_contractions(line)
#         print(line)
        file.write(line)
    file.close()    

cleantext('mimicdemo.train')

dataset = nlp.data.CorpusDataset(
    'mimicdemo.train.clean.txt',
#     sample_splitter=nltk.tokenize.sent_tokenize,
    tokenizer=spacy_tokenizer
#     flatten=False,
#     eos='<eos>'
    )

print('# sentences:', len(dataset))
for sentence in dataset[:50]:
    print('# tokens:', len(sentence), sentence[:100])


In [ ]:
textlinedataset = nlp.data.TextLineDataset('mimicdemo.train.clean.txt', encoding='utf8')
# print(textlinedataset[0])

from spacy.lang.en import English 
import re


nlpp = English()
nlpp.add_pipe(nlpp.create_pipe('sentencizer'))
doc = nlpp(textlinedataset[0])
file = open('mimicdemo.train.final.txt','w+') 
for sent in doc.sents:
    sent = re.sub(r'[^\w\s]', '', sent.text, re.UNICODE)
    sent = re.sub(r'[^\w\s]', '', sent, re.UNICODE)
    sent = re.sub(r'[^\w\s]', '', sent, re.UNICODE)

    sent = re.sub(r'[^\w\s]', '', sent, re.UNICODE)

    sent = re.sub(r'[^\w\s]', '', sent, re.UNICODE)

    sent = re.sub(r'[^\w\s]', '', sent, re.UNICODE)

    sent = re.sub(r'[^\w\s]', '', sent, re.UNICODE)


    sent = re.sub(r'[^\w\s]', '', sent, re.UNICODE)
    sent = re.sub(r'\b(\w+)( \1\b)+', r'\1', sent)
    sent = sent.lower()
    file.write(sent + "\n")
    print(sent)
    

In [ ]:
spacy_tokenizer = nlp.data.SpacyTokenizer()

dataset = nlp.data.CorpusDataset(
    'mimicdemo.train.final.txt',
#     sample_splitter=nltk.tokenize.sent_tokenize,
    tokenizer=spacy_tokenizer
#     flatten=False,
#     eos='<eos>'
    )

print('# sentences:', len(dataset))
for sentence in dataset[:50]:
    print('# tokens:', len(sentence), sentence[:100])


In [ ]:

import warnings
warnings.filterwarnings('ignore')

import itertools
import time
import math
import logging
import random

import mxnet as mx
import gluonnlp as nlp
import numpy as np
from scipy import stats

context = mx.cpu()  # Enable this to run on CPU
# context = mx.gpu(0)  # Enable this to run on GPU

text8 = dataset
counter = nlp.data.count_tokens(itertools.chain.from_iterable(text8))
vocab = nlp.Vocab(counter, unknown_token=None, padding_token=None,
                  bos_token=None, eos_token=None, min_freq=10)
idx_to_counts = [counter[w] for w in vocab.idx_to_token]

def code(sentence):
    return [vocab[token] for token in sentence if token in vocab]

text8 = text8.transform(code, lazy=False)

print('# sentences:', len(text8))
for sentence in text8[:3]:
    print('# tokens:', len(sentence), sentence[:5])

In [ ]:
from data import transform_data_fasttext
batch_size=4096
data = nlp.data.SimpleDataStream([text8])  # input is a stream of datasets, here just 1. Allows scaling to larger corpora that don't fit in memory
data, batchify_fn, subword_function = transform_data_fasttext(
    data, vocab, idx_to_counts, cbow=False, ngrams=[3,4,5,6], ngram_buckets=100000, batch_size=batch_size, window_size=5)

In [ ]:
batches = data.transform(batchify_fn)


In [ ]:
idx_to_subwordidxs = subword_function(vocab.idx_to_token)
for word, subwords in zip(vocab.idx_to_token[:3], idx_to_subwordidxs[:3]):
    print('<'+word+'>', subwords, sep = '\t')

In [ ]:
from model import SG as SkipGramNet

emsize = 300
num_negatives = 5

negatives_weights = mx.nd.array(idx_to_counts)
embedding = SkipGramNet(
    vocab.token_to_idx, emsize, batch_size, negatives_weights, subword_function, num_negatives=5, smoothing=0.75)
embedding.initialize(ctx=context)
embedding.hybridize()
trainer = mx.gluon.Trainer(embedding.collect_params(), 'adagrad', dict(learning_rate=0.05))

print(embedding)


In [ ]:
print(SkipGramNet.hybrid_forward.__doc__)


In [ ]:
def norm_vecs_by_row(x):
    return x / (mx.nd.sum(x * x, axis=1) + 1e-10).sqrt().reshape((-1, 1))


def get_k_closest_tokens(vocab, embedding, k, word):
    word_vec = norm_vecs_by_row(embedding[[word]])
    vocab_vecs = norm_vecs_by_row(embedding[vocab.idx_to_token])
    dot_prod = mx.nd.dot(vocab_vecs, word_vec.T)
    indices = mx.nd.topk(
        dot_prod.reshape((len(vocab.idx_to_token), )),
        k=k + 1,
        ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    result = [vocab.idx_to_token[i] for i in indices[1:]]
    print('closest tokens to "%s": %s' % (word, ", ".join(result)))

In [ ]:
example_token = "radiography"
get_k_closest_tokens(vocab, embedding, 10, example_token)

In [ ]:
log_interval = 500

def train_embedding(num_epochs):
    for epoch in range(1, num_epochs + 1):
        start_time = time.time()
        l_avg = 0
        log_wc = 0

        print('Beginnign epoch %d and resampling data.' % epoch)
        for i, batch in enumerate(batches):
            batch = [array.as_in_context(context) for array in batch]
            with mx.autograd.record():
                l = embedding(*batch)
            l.backward()
            trainer.step(1)

            l_avg += l.mean()
            log_wc += l.shape[0]
            if i % log_interval == 0:
                mx.nd.waitall()
                wps = log_wc / (time.time() - start_time)
                l_avg = l_avg.asscalar() / log_interval
                print('epoch %d, iteration %d, loss %.2f, throughput=%.2fK wps'
                      % (epoch, i, l_avg, wps / 1000))
                start_time = time.time()
                log_wc = 0
                l_avg = 0

        get_k_closest_tokens(vocab, embedding, 10, example_token)
        print("")

In [ ]:
train_embedding(num_epochs=1)
